In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# file_name = "CSE_Netflow.csv"
file_name = "Self-Collected-NF-CSE-CICIDS.parquet"
data = pd.read_parquet(file_name)

In [3]:
data.Label.value_counts()

Label
0    636608
1    349522
Name: count, dtype: int64

In [4]:
data.groupby(by='Attack').count()

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,636608,636608,636608,636608,636608,636608,636608,636608,636608,636608,...,636608,636608,636608,636608,636608,636608,636608,636608,636608,636608
Brute Force -Web,29391,29391,29391,29391,29391,29391,29391,29391,29391,29391,...,29391,29391,29391,29391,29391,29391,29391,29391,29391,29391
Brute Force -XSS,16792,16792,16792,16792,16792,16792,16792,16792,16792,16792,...,16792,16792,16792,16792,16792,16792,16792,16792,16792,16792
DoS attacks-GoldenEye,97869,97869,97869,97869,97869,97869,97869,97869,97869,97869,...,97869,97869,97869,97869,97869,97869,97869,97869,97869,97869
DoS attacks-Hulk,57463,57463,57463,57463,57463,57463,57463,57463,57463,57463,...,57463,57463,57463,57463,57463,57463,57463,57463,57463,57463
DoS attacks-SlowHTTPTest,33922,33922,33922,33922,33922,33922,33922,33922,33922,33922,...,33922,33922,33922,33922,33922,33922,33922,33922,33922,33922
DoS attacks-Slowloris,51557,51557,51557,51557,51557,51557,51557,51557,51557,51557,...,51557,51557,51557,51557,51557,51557,51557,51557,51557,51557
FTP-BruteForce,7202,7202,7202,7202,7202,7202,7202,7202,7202,7202,...,7202,7202,7202,7202,7202,7202,7202,7202,7202,7202
Infilteration,50000,50000,50000,50000,50000,50000,50000,50000,50000,50000,...,50000,50000,50000,50000,50000,50000,50000,50000,50000,50000


In [5]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [6]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [7]:
data.Attack.unique()

array(['DoS attacks-GoldenEye', 'Benign', 'FTP-BruteForce',
       'DoS attacks-Hulk', 'Infilteration', 'DoS attacks-SlowHTTPTest',
       'DoS attacks-Slowloris', 'SQL Injection', 'SSH-Bruteforce',
       'Brute Force -Web', 'Brute Force -XSS'], dtype=object)

In [ ]:
# Scale the dataset based on your needs (machine capacity) ideally ~500k rows

# data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)
data_attack = data[data['Label'] == 1]
data_benign = data[data['Label'] == 0].sample(frac=0.2, random_state=13)
data = pd.concat([data_attack, data_benign], axis=0)
data = data.sample(frac=0.5, random_state=13).reset_index(drop=True)
data.Label.value_counts()

Label
1    174868
0     63554
Name: count, dtype: int64

In [9]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,63554,63554,63554,63554,63554,63554,63554,63554,63554,63554,...,63554,63554,63554,63554,63554,63554,63554,63554,63554,63554
Brute Force -Web,14510,14510,14510,14510,14510,14510,14510,14510,14510,14510,...,14510,14510,14510,14510,14510,14510,14510,14510,14510,14510
Brute Force -XSS,8403,8403,8403,8403,8403,8403,8403,8403,8403,8403,...,8403,8403,8403,8403,8403,8403,8403,8403,8403,8403
DoS attacks-GoldenEye,49023,49023,49023,49023,49023,49023,49023,49023,49023,49023,...,49023,49023,49023,49023,49023,49023,49023,49023,49023,49023
DoS attacks-Hulk,28674,28674,28674,28674,28674,28674,28674,28674,28674,28674,...,28674,28674,28674,28674,28674,28674,28674,28674,28674,28674
DoS attacks-SlowHTTPTest,17043,17043,17043,17043,17043,17043,17043,17043,17043,17043,...,17043,17043,17043,17043,17043,17043,17043,17043,17043,17043
DoS attacks-Slowloris,25787,25787,25787,25787,25787,25787,25787,25787,25787,25787,...,25787,25787,25787,25787,25787,25787,25787,25787,25787,25787
FTP-BruteForce,3655,3655,3655,3655,3655,3655,3655,3655,3655,3655,...,3655,3655,3655,3655,3655,3655,3655,3655,3655,3655
Infilteration,25097,25097,25097,25097,25097,25097,25097,25097,25097,25097,...,25097,25097,25097,25097,25097,25097,25097,25097,25097,25097


In [10]:
X = data.drop(columns=["Attack", "Label"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

In [11]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [12]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [13]:
# (Modified)
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()
X_train['id'] = X_train.index

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()
X_test['id'] = X_test.index

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [14]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h,id
107317,145.82.199.44,182.129.67.88,8.380256e-09,1.049606e-08,0.000990,6.512822e-07,0.000000,0.000000e+00,1.141009e-08,1.141009e-08,...,0.000000,0.000000,8.380256e-09,8.380256e-09,8.380256e-09,8.380256e-09,-1.142600e-08,8.508713e-09,"[8.380256498754423e-09, 1.0496055358705104e-08...",107317
19795,103.57.14.23,182.129.67.88,7.203033e-09,7.035117e-09,0.000002,2.946278e-08,0.000000,0.000000e+00,8.706020e-09,8.700784e-09,...,0.000005,0.000000,7.203033e-09,7.203033e-09,7.203033e-09,7.203033e-09,-9.820928e-09,7.313445e-09,"[7.203033491054604e-09, 7.035116647613036e-09,...",19795
64122,145.82.199.44,190.201.33.51,3.926327e-07,3.834797e-07,0.000557,2.676662e-06,0.000770,1.070665e-06,2.481116e-07,2.532600e-07,...,0.000307,0.000268,3.926327e-07,3.926327e-07,3.926327e-07,3.926327e-07,-5.353325e-07,3.986512e-07,"[3.926327369086904e-07, 3.834796974433861e-07,...",64122
7965,145.82.199.44,190.201.33.51,2.586891e-08,2.526585e-08,0.000009,1.058124e-07,0.000005,7.054158e-08,1.634702e-08,1.668623e-08,...,0.000018,0.000018,2.586891e-08,2.586891e-08,2.586891e-08,2.586891e-08,-3.527079e-08,2.626544e-08,"[2.5868909498806124e-08, 2.526585446210392e-08...",7965
205381,103.57.14.23,190.201.33.51,7.230443e-08,7.061887e-08,0.000038,4.929149e-07,0.000016,1.971660e-07,7.816677e-08,7.701239e-08,...,0.006333,0.006424,7.230443e-08,7.230443e-08,7.230443e-08,7.230443e-08,-9.858299e-08,7.341275e-08,"[7.230442827011002e-08, 7.0618870181696e-08, 3...",205381


In [15]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [16]:
# Training graph (Modified)

train['id'] = train.index

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

test['id'] = test.index

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
# print(test_g)
test_g = test_g.to_directed()
# print(test_g)
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [17]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [18]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [19]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [20]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [21]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [22]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [23]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [24]:
# Convert to GPU
train_g = train_g.to('cuda')

In [ ]:
cnt_wait = 0
best = 1e9
best_t = 0
dur = []
node_features = train_g.ndata['h'] 
edge_features = train_g.edata['h']

for epoch in range(epochs):
    dgi.train()
    if epoch >= 3:
        t0 = time.time()

    dgi_optimizer.zero_grad()
    loss = dgi(train_g, node_features, edge_features)
    loss.backward()
    dgi_optimizer.step()

    if loss < best:
        best = loss
        best_t = epoch
        cnt_wait = 0
        torch.save(dgi.state_dict(), 'best_dgi_CSE_self_collected.pkl')
    else:
        cnt_wait += 1

  # if cnt_wait == patience:
  #     print('Early stopping!')
  #     break

    if epoch >= 3:
        dur.append(time.time() - t0)

    if epoch % 50 == 0:

        print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
            "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
              loss.item(),
              train_g.num_edges() / np.mean(dur) / 1000))

/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Time(s) nan | Loss 1.4200 | ETputs(KTEPS) nan
Epoch 00050 | Time(s) 0.3123 | Loss 1.3968 | ETputs(KTEPS) 1068.68
Epoch 00100 | Time(s) 0.3124 | Loss 1.3863 | ETputs(KTEPS) 1068.34
Epoch 00150 | Time(s) 0.3126 | Loss 1.3863 | ETputs(KTEPS) 1067.76
Epoch 00200 | Time(s) 0.3127 | Loss 1.3863 | ETputs(KTEPS) 1067.44
Epoch 00250 | Time(s) 0.3128 | Loss 1.3863 | ETputs(KTEPS) 1067.03
Epoch 00300 | Time(s) 0.3130 | Loss 1.3863 | ETputs(KTEPS) 1066.40
Epoch 00350 | Time(s) 0.3131 | Loss 1.3863 | ETputs(KTEPS) 1065.95
Epoch 00400 | Time(s) 0.3133 | Loss 1.3863 | ETputs(KTEPS) 1065.54
Epoch 00450 | Time(s) 0.3134 | Loss 1.3863 | ETputs(KTEPS) 1065.14
Epoch 00500 | Time(s) 0.3134 | Loss 1.3863 | ETputs(KTEPS) 1064.90
Epoch 00550 | Time(s) 0.3135 | Loss 1.3863 | ETputs(KTEPS) 1064.64
Epoch 00600 | Time(s) 0.3136 | Loss 1.3863 | ETputs(KTEPS) 1064.39
Epoch 00650 | Time(s) 0.3136 | Loss 1.3863 | ETputs(KTEPS) 1064.23
Epoch 00700 | Time(s) 0.3137 | Loss 1.3863 | ETputs(KTEPS) 1064.11
Ep

In [ ]:
dgi.load_state_dict(torch.load('best_dgi_CSE_self_collected.pkl'))

<All keys matched successfully>

In [27]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [28]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [29]:
# Convert to GPU
test_g = test_g.to('cuda')

In [30]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [31]:
# Multimodal (Fusion) Learning

df_train = pd.DataFrame(training_emb,)
# map the id to the original data
df_train['id'] = train_g.edata['id'].detach().cpu().numpy()


df_raw_train = pd.DataFrame(X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_fuse_train = pd.merge(df_train, df_raw_train, on='id', how='left')
df_fuse_train = df_fuse_train.drop(columns=["id"])
df_fuse_train["Attacks"] = train_g.edata['Attack'].detach().cpu().numpy()
df_fuse_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb,)
# map the id to the original data
df_test['id'] = test_g.edata['id'].detach().cpu().numpy()

df_raw_test = pd.DataFrame(X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_raw_test = pd.merge(df_test, df_raw_test, on='id', how='left')
df_fuse_test = df_raw_test.drop(columns=["id"])
df_fuse_test["Attacks"] = test_g.edata['Attack'].detach().cpu().numpy()
df_fuse_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

# Embeddings CBLOF  Embeddings

In [32]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
from pyod.models.cblof import CBLOF
import gc

from tqdm import tqdm
import itertools

In [33]:
benign_fuse_train_samples = df_fuse_train[df_fuse_train.Label == 0].drop(columns=["Label", "Attacks"])
normal_fuse_train_samples = df_fuse_train.drop(columns=["Label", "Attacks"])

fuse_train_labels = df_fuse_train["Label"]
fuse_test_labels = df_fuse_test["Label"]

fuse_test_samples = df_fuse_test.drop(columns=["Label", "Attacks"])

In [34]:
fuse_test_samples

,0,1,2,3,4,5,6,7,8,9,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
0,-0.093841,0.016038,-0.030644,0.054135,0.203464,0.002625,0.000421,0.026735,0.084260,0.011204,...,8.914575e-07,0.0,0.000255,0.000223,3.269140e-07,3.269140e-07,3.269140e-07,3.269140e-07,-4.457287e-07,3.319251e-07
1,-0.093841,0.016038,-0.030644,0.054135,0.203464,0.002625,0.000421,0.026735,0.084260,0.011204,...,0.000000e+00,0.0,0.000946,0.000000,1.080455e-08,1.080455e-08,1.080455e-08,1.080455e-08,-1.473138e-08,1.097016e-08
2,-0.093841,0.016038,-0.030644,0.054135,0.203464,0.002625,0.000421,0.026735,0.084260,0.011204,...,0.000000e+00,0.0,0.000020,0.000000,2.881213e-08,2.881213e-08,2.881213e-08,2.881213e-08,-3.928371e-08,2.925378e-08
3,-0.093841,0.016038,-0.030644,0.054135,0.203464,0.002625,0.000421,0.026735,0.084260,0.011204,...,0.000000e+00,0.0,0.000179,0.000000,1.282714e-07,1.282714e-07,1.282714e-07,1.282714e-07,-1.748907e-07,1.302376e-07
4,-0.093841,0.016038,-0.030644,0.054135,0.203464,0.002625,0.000421,0.026735,0.084260,0.011204,...,0.000000e+00,0.0,0.050100,0.050899,7.319806e-05,7.319806e-05,7.319806e-05,7.319806e-05,-9.980140e-05,7.432008e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143049,-0.093514,0.027680,-0.055948,0.045657,0.217284,0.010354,0.006881,0.023755,0.108654,-0.007281,...,0.000000e+00,0.0,0.668770,0.000000,7.635443e-06,7.635443e-06,7.635443e-06,7.635443e-06,-1.041049e-05,7.752483e-06
143050,-0.093514,0.027680,-0.055948,0.045657,0.217284,0.010354,0.006881,0.023755,0.108654,-0.007281,...,0.000000e+00,0.0,0.758883,0.000000,8.664277e-06,8.664277e-06,8.664277e-06,8.664277e-06,-1.181325e-05,8.797087e-06
143051,-0.093514,0.027680,-0.055948,0.045657,0.217284,0.010354,0.006881,0.023755,0.108654,-0.007281,...,0.000000e+00,0.0,0.105023,0.000000,1.175371e-06,1.175371e-06,1.175371e-06,1.175371e-06,-1.602551e-06,1.193387e-06
143052,-0.093514,0.027680,-0.055948,0.045657,0.217284,0.010354,0.006881,0.023755,0.108654,-0.007281,...,0.000000e+00,0.0,0.003785,0.000000,4.321789e-08,4.321789e-08,4.321789e-08,4.321789e-08,-5.892514e-08,4.388036e-08


In [35]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [103]:
n_est = [5,6,7,9,10] # cant be lower than 5 or 4
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(benign_fuse_train_samples)
    except Exception as e:
        print(n_est)
        continue
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [05:39<00:00, 11.32s/it]

{'n_estimators': 10, 'con': 0.05}
0.632313073914295
              precision    recall  f1-score   support

           0     0.9237    0.9497    0.9366    381966
           1     0.3846    0.2860    0.3281     41942

    accuracy                         0.8841    423908
   macro avg     0.6542    0.6179    0.6323    423908
weighted avg     0.8704    0.8841    0.8764    423908



In [104]:
n_est = [5,6,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(normal_fuse_train_samples)
    except Exception as e:
        print(n_est)
        continue
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

 13%|█▎        | 4/30 [00:36<03:31,  8.15s/it]

5


100%|██████████| 30/30 [05:29<00:00, 10.99s/it]

{'n_estimators': 9, 'con': 0.2}
0.5451412481676391
              precision    recall  f1-score   support

           0     0.9188    0.8144    0.8634    381966
           1     0.1692    0.3441    0.2268     41942

    accuracy                         0.7679    423908
   macro avg     0.5440    0.5793    0.5451    423908
weighted avg     0.8446    0.7679    0.8005    423908



In [105]:
# HBOS  Embeddings+Raw (Multimodal)

In [106]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [107]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

 19%|█▉        | 7/36 [01:18<05:25, 11.22s/it]


KeyboardInterrupt: 

In [ ]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:52<00:00,  1.46s/it]

{'n_estimators': 5, 'con': 0.01}
0.4893857068710194
              precision    recall  f1-score   support

           0     0.6379    0.6315    0.6347     38196
           1     0.3410    0.3472    0.3441     20972

    accuracy                         0.5307     59168
   macro avg     0.4895    0.4894    0.4894     59168
weighted avg     0.5327    0.5307    0.5317     59168



In [ ]:
##  PCA  Emb+Raw (Multimodal/Fusion) Learning

In [ ]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:50<00:00,  1.41s/it]

{'n_estimators': 25, 'con': 0.04}
0.4521499568535427
              precision    recall  f1-score   support

           0     0.6238    0.3855    0.4765     38196
           1     0.3400    0.5766    0.4278     20972

    accuracy                         0.4532     59168
   macro avg     0.4819    0.4810    0.4521     59168
weighted avg     0.5232    0.4532    0.4592     59168



In [ ]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [01:01<00:00,  1.72s/it]

{'n_estimators': 5, 'con': 0.05}
0.43776217172464377
              precision    recall  f1-score   support

           0     0.5956    0.5155    0.5526     38196
           1     0.2911    0.3624    0.3229     20972

    accuracy                         0.4612     59168
   macro avg     0.4433    0.4390    0.4378     59168
weighted avg     0.4877    0.4612    0.4712     59168



In [ ]:
##  IF  Emb+Raw

In [ ]:
for i in range(len(benign_fuse_train_samples.columns)):
    benign_fuse_train_samples.rename(columns={benign_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(normal_fuse_train_samples.columns)):
    normal_fuse_train_samples.rename(columns={normal_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(fuse_test_samples.columns)):
    fuse_test_samples.rename(columns={fuse_test_samples.columns[i]: f"feature {i}"}, inplace=True)

In [ ]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples.to_numpy())
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]

/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  4%|▍         | 1/24 [00:00<00:11,  2.09it/s]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  8%|▊         | 2/24 [00:00<00:09,  2.30it/s]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 12%|█▎        | 3/24 [00:01<00:08,  2.37it/s]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warni

{'n_estimators': 100, 'con': 0.01}
0.39985799202747224
              precision    recall  f1-score   support

           0     0.6452    0.9885    0.7807     38196
           1     0.3183    0.0098    0.0190     20972

    accuracy                         0.6416     59168
   macro avg     0.4817    0.4991    0.3999     59168
weighted avg     0.5293    0.6416    0.5107     59168



In [ ]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:20<00:00,  1.15it/s]

{'n_estimators': 150, 'con': 0.01}
0.4000915949078085
              precision    recall  f1-score   support

           0     0.6445    0.9844    0.7790     38196
           1     0.2797    0.0110    0.0212     20972

    accuracy                         0.6394     59168
   macro avg     0.4621    0.4977    0.4001     59168
weighted avg     0.5152    0.6394    0.5104     59168



## Supervised attack classification (Fusion)

We now train a supervised classifier on the fused features to predict multi-class attack labels:
- Features: embeddings + raw numeric features from `df_fuse_train`/`df_fuse_test` (without `Label`, `Attacks`).
- Target: `Attacks` (encoded integer classes from earlier `LabelEncoder`).
- Model: HistGradientBoostingClassifier (fast, strong on tabular data). Class imbalance handled via per-sample weights.
- Metrics: macro F1, per-class report, and confusion matrix.

In [36]:
# Prepare supervised train/test for attack classification
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.utils import class_weight
import pickle

# Build train features/targets from already prepared fused DataFrames
X_sup_train = df_fuse_train.drop(columns=["Label", "Attacks"]).copy()
y_sup_train = df_fuse_train["Attacks"].copy()

X_sup_test = df_fuse_test.drop(columns=["Label", "Attacks"]).copy()
y_sup_test = df_fuse_test["Attacks"].copy()

# Compute sample weights to mitigate class imbalance on training set
classes = np.unique(y_sup_train)
class_w = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes, y=y_sup_train
)
class_to_w = {c: w for c, w in zip(classes, class_w)}
sample_weight = y_sup_train.map(class_to_w).values

# Feature names to all str
X_sup_train.columns = X_sup_train.columns.map(str)
X_sup_test.columns = X_sup_test.columns.map(str)

## Classification Methods (Fused Features)

Now we'll compare multiple classification algorithms on the fused features:

In [37]:
# Random Forest with Grid Search
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST CLASSIFIER (Fused Features)")
print("="*60)

try:
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [10, 20, 30]
    params = list(itertools.product(n_estimators_grid, max_depth_grid))

    score = -1
    best_params = None
    best_model = None

    print(f"Testing {len(params)} configurations...")

    for n_est, depth in tqdm(params):
        try:
            rf_clf = RandomForestClassifier(
                n_estimators=n_est,
                max_depth=depth,
                class_weight='balanced',
                random_state=13,
                n_jobs=-1
            )
            
            rf_clf.fit(X_sup_train, y_sup_train)
            y_pred_rf = rf_clf.predict(X_sup_test)
            rf_f1 = f1_score(y_sup_test, y_pred_rf, average='macro')
            
            if rf_f1 > score:
                score = rf_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth
                }
                best_model = rf_clf
                # Save best model
                with open('best_rf_classifier_fused.pkl', 'wb') as f:
                    pickle.dump(rf_clf, f)
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}): {str(e)}")
            continue

    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_rf_classifier_fused.pkl\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed.")
        
except Exception as e:
    print(f"Unexpected error in Random Forest: {str(e)}")

RANDOM FOREST CLASSIFIER (Fused Features)
Testing 9 configurations...


100%|██████████| 9/9 [05:17<00:00, 35.28s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 30}
Best Macro F1: 0.8009
Model saved to: best_rf_classifier_fused.pkl

              precision    recall  f1-score   support

           0     0.9604    0.9487    0.9545     38132
           1     0.7617    0.9584    0.8488      8706
           2     0.6714    0.6501    0.6606      5042
           3     0.8349    0.6868    0.7536     29414
           4     1.0000    1.0000    1.0000     17204
           5     0.3836    0.5801    0.4618     10226
           6     0.6436    0.5827    0.6116     15472
           7     0.9877    0.8769    0.9290      2194
           8     1.0000    0.9995    0.9998     15058
           9     0.7393    0.7956    0.7664      1458
          10     0.9813    0.7095    0.8235       148

    accuracy                         0.8276    143054
   macro avg     0.8149    0.7989    0.8009    143054
weighted avg     0.8439    0.8276    0.8319    143054



In [38]:
# XGBoost with Grid Search
try:
    from xgboost import XGBClassifier
    import pickle
    
    print("="*60)
    print("XGBOOST CLASSIFIER (Fused Features)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        try:
            xgb_clf = XGBClassifier(
                n_estimators=n_est,
                max_depth=depth,
                learning_rate=lr,
                random_state=13,
                tree_method='hist', 
                device="cuda"
            )
            
            xgb_clf.fit(X_sup_train, y_sup_train, sample_weight=sample_weight)
            y_pred_xgb = xgb_clf.predict(X_sup_test)
            xgb_f1 = f1_score(y_sup_test, y_pred_xgb, average='macro')
            
            if xgb_f1 > score:
                score = xgb_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth,
                    'learning_rate': lr
                }
                best_model = xgb_clf
                # Save best model
                xgb_clf.save_model('best_xgb_classifier_fused.json')
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_xgb_classifier_fused.json\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed. XGBoost might not support your GPU.")
    
except ImportError:
    print("XGBoost not installed. Install with: pip install xgboost")
except Exception as e:
    print(f"Unexpected error: {str(e)}")

XGBOOST CLASSIFIER (Fused Features)
Testing 27 configurations...


  0%|          | 0/27 [00:00<?, ?it/s]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [12:04:04] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
100%|██████████| 27/27 [21:42<00:00, 48.24s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 8, 'learning_rate': 0.1}
Best Macro F1: 0.8174
Model saved to: best_xgb_classifier_fused.json

              precision    recall  f1-score   support

           0     0.9632    0.9604    0.9618     38132
           1     0.8167    0.9257    0.8678      8706
           2     0.6247    0.7515    0.6823      5042
           3     0.9258    0.6528    0.7657     29414
           4     0.9999    1.0000    0.9999     17204
           5     0.3879    0.6467    0.4849     10226
           6     0.6301    0.6061    0.6179     15472
           7     0.9893    0.9230    0.9550      2194
           8     1.0000    1.0000    1.0000     15058
           9     0.6479    0.8772    0.7453      1458
          10     0.9236    0.8986    0.9110       148

    accuracy                         0.8344    143054
   macro avg     0.8099    0.8402    0.8174    143054
weighted avg     0.8629    0.8344    0.8409    143054



In [39]:
# CatBoost with Grid Search (Expanded, no l2_leaf_reg/border_count, fixed best score logic)
try:
    from catboost import CatBoostClassifier
    
    print("Starting CatBoost Grid Search (Expanded)...")
    
    # Expanded hyperparameter grid
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    print(f"Testing {len(params)} configurations...")
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'  # Use 'CPU' if GPU not available
            )
            
            cat_clf.fit(X_sup_train, y_sup_train)
            y_pred_cat = cat_clf.predict(X_sup_test)
            cat_f1 = f1_score(y_sup_test, y_pred_cat, average='macro')
            
            if cat_f1 > best_score:
                best_score = cat_f1
                best_params = {
                    'iterations': iterations,
                    'depth': depth,
                    'learning_rate': lr
                }
                best_model = cat_clf
                # Save the best model with unique name for fused features
                best_model_path = "best_catboost_classifier_fused.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\n" + "="*60)
    print("BEST CATBOOST HYPERPARAMETERS (Fused Features):")
    print(best_params)
    print(f"Best Macro F1: {best_score:.4f}")
    print("="*60)
    if best_model_path:
        print(f"\nBest CatBoost model saved to: {best_model_path}")
    
    # Final evaluation
    y_pred_best = best_model.predict(X_sup_test)
    print("\n" + "="*60)
    print("FINAL CLASSIFICATION REPORT (Best CatBoost - Fused):")
    print("="*60)
    print(classification_report(y_sup_test, y_pred_best, digits=4))
    
except ImportError:
    print("CatBoost not installed. Install with: pip install catboost")

Starting CatBoost Grid Search (Expanded)...
Testing 27 configurations...


100%|██████████| 27/27 [07:01<00:00, 15.62s/it]


BEST CATBOOST HYPERPARAMETERS (Fused Features):
{'iterations': 500, 'depth': 8, 'learning_rate': 0.1}
Best Macro F1: 0.7692

Best CatBoost model saved to: best_catboost_classifier_fused.cbm

FINAL CLASSIFICATION REPORT (Best CatBoost - Fused):
              precision    recall  f1-score   support

           0     0.9774    0.9475    0.9622     38132
           1     0.5668    0.9444    0.7084      8706
           2     0.5250    0.7077    0.6028      5042
           3     0.9261    0.5330    0.6766     29414
           4     0.9999    1.0000    0.9999     17204
           5     0.3978    0.6468    0.4926     10226
           6     0.6450    0.6011    0.6223     15472
           7     0.9930    0.9079    0.9486      2194
           8     0.9999    1.0000    0.9999     15058
           9     0.5702    0.9026    0.6989      1458
          10     0.6147    0.9595    0.7493       148

    accuracy                         0.8055    143054
   macro avg     0.7469    0.8319    0.7692    1430

In [40]:
# Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES CLASSIFIER (Fused Features)")
print("="*60)

try:
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [10, 20, 30]

    params = list(itertools.product(n_estimators_grid, max_depth_grid))

    score = -1
    best_params = None
    best_model = None

    print(f"Testing {len(params)} configurations...")

    for n_est, depth in tqdm(params):
        try:
            et_clf = ExtraTreesClassifier(
                n_estimators=n_est,
                max_depth=depth,
                class_weight='balanced',
                random_state=13,
                n_jobs=-1
            )
            
            et_clf.fit(X_sup_train, y_sup_train)
            y_pred_et = et_clf.predict(X_sup_test)
            et_f1 = f1_score(y_sup_test, y_pred_et, average='macro')
            
            if et_f1 > score:
                score = et_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth
                }
                best_model = et_clf
                # Save best model
                with open('best_et_classifier_fused.pkl', 'wb') as f:
                    pickle.dump(et_clf, f)
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}): {str(e)}")
            continue

    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_et_classifier_fused.pkl\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed.")
        
except Exception as e:
    print(f"Unexpected error in Extra Trees: {str(e)}")

EXTRA TREES CLASSIFIER (Fused Features)
Testing 9 configurations...


100%|██████████| 9/9 [03:49<00:00, 25.51s/it]



Best Parameters: {'n_estimators': 100, 'max_depth': 30}
Best Macro F1: 0.6911
Model saved to: best_et_classifier_fused.pkl

              precision    recall  f1-score   support

           0     0.9367    0.9039    0.9200     38132
           1     0.4624    0.9150    0.6144      8706
           2     0.5263    0.6656    0.5878      5042
           3     0.7532    0.2664    0.3936     29414
           4     1.0000    0.9999    0.9999     17204
           5     0.2525    0.6657    0.3661     10226
           6     0.6318    0.5217    0.5715     15472
           7     0.9886    0.6691    0.7980      2194
           8     0.9999    0.7872    0.8809     15058
           9     0.6158    0.7894    0.6919      1458
          10     0.9174    0.6757    0.7782       148

    accuracy                         0.7010    143054
   macro avg     0.7350    0.7145    0.6911    143054
weighted avg     0.7855    0.7010    0.7053    143054



## Raw Features Classification (Comparison Baseline)

Now we'll train classifiers on **raw features only** (without graph embeddings) to compare the benefit of multimodal fusion:

In [41]:
# Prepare raw features (without embeddings)
# Use only the original numeric features from X_train/X_test

X_raw_train = X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h", "id"]).copy()
y_raw_train = y_train["Attack"].copy()

X_raw_test = X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h", "id"]).copy()
y_raw_test = y_test["Attack"].copy()

# Encode labels
y_raw_train_encoded = lab_enc.transform(y_train["Attack"])
y_raw_test_encoded = lab_enc.transform(y_test["Attack"])

# Compute sample weights
classes_raw = np.unique(y_raw_train_encoded)
class_w_raw = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes_raw, y=y_raw_train_encoded
)
class_to_w_raw = {c: w for c, w in zip(classes_raw, class_w_raw)}
sample_weight_raw = pd.Series(y_raw_train_encoded).map(class_to_w_raw).values

# Feature names to str
X_raw_train.columns = X_raw_train.columns.map(str)
X_raw_test.columns = X_raw_test.columns.map(str)

print(f"Raw feature shape - Train: {X_raw_train.shape}, Test: {X_raw_test.shape}")
print(f"Number of classes: {len(np.unique(y_raw_train_encoded))}")

Raw feature shape - Train: (166895, 39), Test: (71527, 39)
Number of classes: 11


In [42]:
# Raw Features - Random Forest
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST (Raw Features Only)")

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]
params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    rf_clf = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    rf_clf.fit(X_raw_train, y_raw_train_encoded)
    y_pred = rf_clf.predict(X_raw_test)
    f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = rf_clf
        # Save best model
        with open('best_rf_classifier_raw.pkl', 'wb') as f:
            pickle.dump(rf_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_rf_classifier_raw.pkl\n")
print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))

RANDOM FOREST (Raw Features Only)
Testing 9 configurations...


100%|██████████| 9/9 [01:35<00:00, 10.60s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 30}
Best Macro F1: 0.8325
Model saved to: best_rf_classifier_raw.pkl

              precision    recall  f1-score   support

           0     0.9861    0.9560    0.9708     19066
           1     0.9070    0.9302    0.9185      4353
           2     0.8229    0.7703    0.7957      2521
           3     0.8519    0.8654    0.8586     14707
           4     0.9999    1.0000    0.9999      8602
           5     0.4479    0.6294    0.5234      5113
           6     0.7173    0.5379    0.6148      7736
           7     0.9701    0.9754    0.9727      1097
           8     0.9996    0.9997    0.9997      7529
           9     0.6381    0.8491    0.7287       729
          10     0.7209    0.8378    0.7750        74

    accuracy                         0.8697     71527
   macro avg     0.8238    0.8501    0.8325     71527
weighted avg     0.8794    0.8697    0.8718     71527



In [43]:
# Raw Features - XGBoost
try:
    from xgboost import XGBClassifier
    
    print("="*60)
    print("XGBOOST (Raw Features Only)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        xgb_clf = XGBClassifier(
            n_estimators=n_est,
            max_depth=depth,
            learning_rate=lr,
            random_state=13,
            tree_method='hist',
            device='cuda'
        )
        
        xgb_clf.fit(X_raw_train, y_raw_train_encoded, sample_weight=sample_weight_raw)
        y_pred = xgb_clf.predict(X_raw_test)
        f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
        
        if f1 > score:
            score = f1
            best_params = {'n_estimators': n_est, 'max_depth': depth, 'learning_rate': lr}
            best_model = xgb_clf
            # Save best model
            xgb_clf.save_model('best_xgb_classifier_raw.json')
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {score:.4f}")
    print("Model saved to: best_xgb_classifier_raw.json\n")
    print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))
    
except ImportError:
    print("XGBoost not installed.")

XGBOOST (Raw Features Only)
Testing 27 configurations...


100%|██████████| 27/27 [04:23<00:00,  9.76s/it]


Best Parameters: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.8480
Model saved to: best_xgb_classifier_raw.json

              precision    recall  f1-score   support

           0     0.9902    0.9601    0.9749     19066
           1     0.8626    0.9228    0.8917      4353
           2     0.7595    0.7541    0.7568      2521
           3     0.9111    0.8542    0.8817     14707
           4     0.9999    1.0000    0.9999      8602
           5     0.4884    0.6773    0.5676      5113
           6     0.6889    0.5911    0.6363      7736
           7     0.9801    0.9891    0.9846      1097
           8     1.0000    1.0000    1.0000      7529
           9     0.6166    0.8848    0.7268       729
          10     0.8846    0.9324    0.9079        74

    accuracy                         0.8773     71527
   macro avg     0.8347    0.8696    0.8480     71527
weighted avg     0.8877    0.8773    0.8805     71527



In [44]:
# Raw Features - CatBoost (Expanded grid, no l2_leaf_reg/border_count)
try:
    from catboost import CatBoostClassifier
    
    print("="*60)
    print("CATBOOST (Raw Features Only, Expanded)")
    print("="*60)
    
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    print(f"Testing {len(params)} configurations...")
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'
            )
            
            cat_clf.fit(X_raw_train, y_raw_train_encoded)
            y_pred = cat_clf.predict(X_raw_test)
            f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
            
            if f1 > best_score:
                best_score = f1
                best_params = {'iterations': iterations, 'depth': depth, 'learning_rate': lr}
                best_model = cat_clf
                # Save the best model with unique name for raw features
                best_model_path = "best_catboost_classifier_raw.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {best_score:.4f}\n")
    if best_model_path:
        print(f"Best CatBoost model saved to: {best_model_path}")
    print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))
    
except ImportError:
    print("CatBoost not installed.")

CATBOOST (Raw Features Only, Expanded)
Testing 27 configurations...


100%|██████████| 27/27 [03:10<00:00,  7.07s/it]


Best Parameters: {'iterations': 500, 'depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.8183

Best CatBoost model saved to: best_catboost_classifier_raw.cbm
              precision    recall  f1-score   support

           0     0.9867    0.9503    0.9682     19066
           1     0.7465    0.9341    0.8298      4353
           2     0.7122    0.7057    0.7089      2521
           3     0.9133    0.8229    0.8657     14707
           4     0.9999    1.0000    0.9999      8602
           5     0.4925    0.6626    0.5650      5113
           6     0.7024    0.6003    0.6473      7736
           7     0.9518    0.9909    0.9710      1097
           8     0.9997    1.0000    0.9999      7529
           9     0.6020    0.8944    0.7196       729
          10     0.5882    0.9459    0.7254        74

    accuracy                         0.8673     71527
   macro avg     0.7905    0.8643    0.8183     71527
weighted avg     0.8793    0.8673    0.8704     71527



In [45]:
# Raw Features - Extra Trees
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES (Raw Features Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]

params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    et_clf = ExtraTreesClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    et_clf.fit(X_raw_train, y_raw_train_encoded)
    y_pred = et_clf.predict(X_raw_test)
    f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = et_clf
        # Save best model
        with open('best_et_classifier_raw.pkl', 'wb') as f:
            pickle.dump(et_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_et_classifier_raw.pkl\n")
print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))

EXTRA TREES (Raw Features Only)
Testing 9 configurations...


100%|██████████| 9/9 [00:50<00:00,  5.58s/it]



Best Parameters: {'n_estimators': 100, 'max_depth': 30}
Best Macro F1: 0.8407
Model saved to: best_et_classifier_raw.pkl

              precision    recall  f1-score   support

           0     0.9897    0.9770    0.9833     19066
           1     0.9172    0.9410    0.9289      4353
           2     0.8161    0.7798    0.7976      2521
           3     0.8932    0.8736    0.8833     14707
           4     1.0000    1.0000    1.0000      8602
           5     0.4798    0.6789    0.5622      5113
           6     0.7375    0.5606    0.6370      7736
           7     0.9737    0.9781    0.9759      1097
           8     0.9997    0.9997    0.9997      7529
           9     0.6170    0.8573    0.7176       729
          10     0.7093    0.8243    0.7625        74

    accuracy                         0.8840     71527
   macro avg     0.8303    0.8609    0.8407     71527
weighted avg     0.8936    0.8840    0.8860     71527



## Embeddings Only Classification (Graph Features)

Now we'll train classifiers on **embeddings only** (graph features without raw features) to isolate the value of graph-based learning:

In [46]:
# Prepare embeddings-only features (graph features without raw data)
# Extract only the embedding columns (first 256 dimensions from graph encoder)

# From the fused dataframes, extract only embedding columns
# df_fuse_train has: [0-255] = embeddings, [256+] = raw features
num_embedding_dims = 256  # Based on the DGI encoder output dimension

X_emb_train = df_fuse_train.iloc[:, :num_embedding_dims].copy()
y_emb_train = df_fuse_train["Attacks"].copy()

X_emb_test = df_fuse_test.iloc[:, :num_embedding_dims].copy()
y_emb_test = df_fuse_test["Attacks"].copy()

# Compute sample weights
classes_emb = np.unique(y_emb_train)
class_w_emb = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes_emb, y=y_emb_train
)
class_to_w_emb = {c: w for c, w in zip(classes_emb, class_w_emb)}
sample_weight_emb = y_emb_train.map(class_to_w_emb).values

# Feature names to str
X_emb_train.columns = X_emb_train.columns.map(str)
X_emb_test.columns = X_emb_test.columns.map(str)

print(f"Embeddings shape - Train: {X_emb_train.shape}, Test: {X_emb_test.shape}")
print(f"Number of classes: {len(np.unique(y_emb_train))}")
print(f"Using only graph embeddings (no raw features)")

Embeddings shape - Train: (333790, 256), Test: (143054, 256)
Number of classes: 11
Using only graph embeddings (no raw features)


In [47]:
# Embeddings Only - Random Forest
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST (Embeddings Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]
params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    rf_clf = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    rf_clf.fit(X_emb_train, y_emb_train)
    y_pred = rf_clf.predict(X_emb_test)
    f1 = f1_score(y_emb_test, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = rf_clf
        # Save best model
        with open('best_rf_classifier_embeddings.pkl', 'wb') as f:
            pickle.dump(rf_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_rf_classifier_embeddings.pkl\n")
print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))

RANDOM FOREST (Embeddings Only)
Testing 9 configurations...


100%|██████████| 9/9 [01:30<00:00, 10.04s/it]



Best Parameters: {'n_estimators': 100, 'max_depth': 10}
Best Macro F1: 0.0221
Model saved to: best_rf_classifier_embeddings.pkl

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000     38132
           1     0.0000    0.0000    0.0000      8706
           2     0.9921    0.0500    0.0952      5042
           3     0.0000    0.0000    0.0000     29414
           4     0.0064    0.0163    0.0092     17204
           5     0.0000    0.0000    0.0000     10226
           6     0.0000    0.0000    0.0000     15472
           7     0.0109    0.2484    0.0210      2194
           8     0.0771    0.2507    0.1179     15058
           9     0.0000    0.0000    0.0000      1458
          10     0.0000    0.0000    0.0000       148

    accuracy                         0.0339    143054
   macro avg     0.0988    0.0514    0.0221    143054
weighted avg     0.0440    0.0339    0.0172    143054



/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

In [48]:
# Embeddings Only - XGBoost
try:
    from xgboost import XGBClassifier
    
    print("="*60)
    print("XGBOOST (Embeddings Only)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        xgb_clf = XGBClassifier(
            n_estimators=n_est,
            max_depth=depth,
            learning_rate=lr,
            random_state=13,
            tree_method='hist',
            device='gpu'
        )
        
        xgb_clf.fit(X_emb_train, y_emb_train, sample_weight=sample_weight_emb)
        y_pred = xgb_clf.predict(X_emb_test)
        f1 = f1_score(y_emb_test, y_pred, average='macro')
        
        if f1 > score:
            score = f1
            best_params = {'n_estimators': n_est, 'max_depth': depth, 'learning_rate': lr}
            best_model = xgb_clf
            # Save best model
            xgb_clf.save_model('best_xgb_classifier_embeddings.json')
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {score:.4f}")
    print("Model saved to: best_xgb_classifier_embeddings.json\n")
    print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))
    
except ImportError:
    print("XGBoost not installed.")

XGBOOST (Embeddings Only)
Testing 27 configurations...


100%|██████████| 27/27 [12:04<00:00, 26.85s/it]



Best Parameters: {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.05}
Best Macro F1: 0.0397
Model saved to: best_xgb_classifier_embeddings.json

              precision    recall  f1-score   support

           0     0.3230    0.1174    0.1722     38132
           1     0.0000    0.0000    0.0000      8706
           2     1.0000    0.0452    0.0865      5042
           3     0.0000    0.0000    0.0000     29414
           4     0.1009    0.7565    0.1781     17204
           5     0.0000    0.0000    0.0000     10226
           6     0.0000    0.0000    0.0000     15472
           7     0.0000    0.0000    0.0000      2194
           8     0.0000    0.0000    0.0000     15058
           9     0.0000    0.0000    0.0000      1458
          10     0.0000    0.0000    0.0000       148

    accuracy                         0.1239    143054
   macro avg     0.1294    0.0836    0.0397    143054
weighted avg     0.1335    0.1239    0.0704    143054



/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

In [49]:
# Embeddings Only - CatBoost (Expanded grid, no l2_leaf_reg/border_count)
try:
    from catboost import CatBoostClassifier
    
    print("="*60)
    print("CATBOOST (Embeddings Only, Expanded)")
    print("="*60)
    
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    print(f"Testing {len(params)} configurations...")
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'
            )
            
            cat_clf.fit(X_emb_train, y_emb_train)
            y_pred = cat_clf.predict(X_emb_test)
            f1 = f1_score(y_emb_test, y_pred, average='macro')
            
            if f1 > best_score:
                best_score = f1
                best_params = {'iterations': iterations, 'depth': depth, 'learning_rate': lr}
                best_model = cat_clf
                # Save the best model with unique name for embeddings
                best_model_path = "best_catboost_classifier_embeddings.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {best_score:.4f}\n")
    if best_model_path:
        print(f"Best CatBoost model saved to: {best_model_path}")
    print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))
    
except ImportError:
    print("CatBoost not installed.")

CATBOOST (Embeddings Only, Expanded)
Testing 27 configurations...


100%|██████████| 27/27 [05:11<00:00, 11.54s/it]


Best Parameters: {'iterations': 200, 'depth': 8, 'learning_rate': 0.01}
Best Macro F1: 0.0439

Best CatBoost model saved to: best_catboost_classifier_embeddings.cbm
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000     38132
           1     0.0000    0.0000    0.0000      8706
           2     0.0412    0.4090    0.0749      5042
           3     0.1579    0.2629    0.1973     29414
           4     0.0058    0.0074    0.0065     17204
           5     0.0000    0.0000    0.0000     10226
           6     0.0000    0.0000    0.0000     15472
           7     0.0000    0.0000    0.0000      2194
           8     0.1717    0.2507    0.2038     15058
           9     0.0000    0.0000    0.0000      1458
          10     0.0000    0.0000    0.0000       148

    accuracy                         0.0957    143054
   macro avg     0.0342    0.0845    0.0439    143054
weighted avg     0.0527    0.0957    0.0654    143054




/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division

In [50]:
# Embeddings Only - Extra Trees
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES (Embeddings Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]

params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    et_clf = ExtraTreesClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    et_clf.fit(X_emb_train, y_emb_train)
    y_pred = et_clf.predict(X_emb_test)
    f1 = f1_score(y_emb_test, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = et_clf
        # Save best model
        with open('best_et_classifier_embeddings.pkl', 'wb') as f:
            pickle.dump(et_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_et_classifier_embeddings.pkl\n")
print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))

EXTRA TREES (Embeddings Only)
Testing 9 configurations...


100%|██████████| 9/9 [01:35<00:00, 10.66s/it]



Best Parameters: {'n_estimators': 100, 'max_depth': 10}
Best Macro F1: 0.0208
Model saved to: best_et_classifier_embeddings.pkl

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000     38132
           1     0.0000    0.0000    0.0000      8706
           2     0.9921    0.0500    0.0952      5042
           3     0.0000    0.0000    0.0000     29414
           4     0.0767    0.5130    0.1334     17204
           5     0.0000    0.0000    0.0000     10226
           6     0.0000    0.0000    0.0000     15472
           7     0.0000    0.0000    0.0000      2194
           8     0.0000    0.0000    0.0000     15058
           9     0.0000    0.0000    0.0000      1458
          10     0.0000    0.0000    0.0000       148

    accuracy                         0.0635    143054
   macro avg     0.0972    0.0512    0.0208    143054
weighted avg     0.0442    0.0635    0.0194    143054



/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

## Summary Comparison Table

Create a comparison table of all methods (Fused vs Raw features):

In [49]:
# Create a summary comparison table
# NOTE: After running all cells above, manually collect the F1 scores and create a comparison

print("="*95)
print("PERFORMANCE COMPARISON: FUSED vs EMBEDDINGS vs RAW FEATURES")
print("="*95)
print("\nInstructions:")
print("1. Run all cells above to get F1 scores for each method")
print("2. Record the best Macro F1 score for each method")
print("3. Compare three approaches:")
print("   - Fused Features: Embeddings + Raw (Multimodal)")
print("   - Embeddings Only: Graph features only")
print("   - Raw Features: Traditional features only")
print("\nExpected format:")
print("-" * 95)
print(f"{'Method':<20} {'Fused (Emb+Raw)':<25} {'Embeddings Only':<25} {'Raw Only':<25}")
print("-" * 95)
print(f"{'Random Forest':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'XGBoost':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'CatBoost':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'Extra Trees':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print("-" * 95)
print("\nAnalysis:")
print("- Fused features should show best performance (multimodal learning)")
print("- Embeddings capture structural/graph patterns")
print("- Raw features provide traditional statistical information")
print("- Compare to understand the contribution of each modality")

PERFORMANCE COMPARISON: FUSED vs EMBEDDINGS vs RAW FEATURES

Instructions:
1. Run all cells above to get F1 scores for each method
2. Record the best Macro F1 score for each method
3. Compare three approaches:
   - Fused Features: Embeddings + Raw (Multimodal)
   - Embeddings Only: Graph features only
   - Raw Features: Traditional features only

Expected format:
-----------------------------------------------------------------------------------------------
Method               Fused (Emb+Raw)           Embeddings Only           Raw Only                 
-----------------------------------------------------------------------------------------------
Random Forest        [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
XGBoost              [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
CatBoost             [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
Extra Trees          [INSERT SCORE]           

## Key Insights & Interpretation

After running all experiments, analyze the results to answer:

1. **Which approach performs best overall?**
   - Fused features (multimodal) should ideally outperform single modalities
   - Compare the magnitude of improvements

2. **What is the value of graph embeddings?**
   - Compare Embeddings-only vs Raw-only to see if graph structure helps
   - If embeddings alone beat raw features, graph learning is beneficial

3. **Is multimodal fusion effective?**
   - Compare Fused vs (Embeddings + Raw separately)
   - Synergy should provide additional gains beyond individual modalities

4. **Which classifier is most suitable?**
   - Identify the best performing algorithm for each feature type
   - Consider computational cost vs accuracy trade-offs

5. **Feature contribution analysis:**
   - If Fused ≈ Embeddings > Raw: Graph structure dominates
   - If Fused ≈ Raw > Embeddings: Traditional features dominate
   - If Fused > Both: True synergy from multimodal learning